In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## setting

In [ ]:
SAMPLE_FILE_PATH = "../input/facial-expression-recognition-challenge/icml_face_data.csv/icml_face_data.csv"


NUM_CLASSES = 7

TRAIN_HDF5 = "/kaggle/working/train.hdf5"
VAL_HDF5 = "/kaggle/working/val.hdf5"
TEST_HDF5 = "/kaggle/working/test.hdf5"

BATCH_SIZE = 128
OUTPUT_PATH = "/kaggle/working"

DATASET_MEAN_FILE = OUTPUT_PATH + "/rgb_mean.json"

MODEL_FILE = OUTPUT_PATH + "/model.h5"


## EpochCheckpoint

In [ ]:
from tensorflow.keras.callbacks import Callback
import os


class EpochCheckpoint(Callback):
    def __init__(self,output_path,every=5,start_at=0):
        super(Callback,self).__init__()

        self.output_path = output_path
        self.every = every
        self.start_epoch = start_at


    def on_epoch_end(self, epoch, logs={}):
        if (self.start_epoch + 1)% self.every ==0:
            p = os.path.sep.join([self.output_path,
                                  "epoch_{}.hdf5".format(self.start_epoch + 1)])
            self.model.save(p, overwrite = True)
            self.start_epoch += 1


## ImageToArrayPreprocessor

In [ ]:
from tensorflow.keras.preprocessing.image import img_to_array

class ImageToArrayPreprocesor:
    def __init__(self,data_format=None):
        self.data_format = data_format

    def preprocess(self,image):
        return img_to_array(image, data_format=self.data_format)

## HDF5DatasetWriter

In [ ]:
import os
import h5py


class HDF5DatasetWriter:
    def __init__(self, dims, output_path, data_key="images", buf_size=1000):
        if os.path.exists(output_path):
            raise ValueError("您提供的输出文件{}已经存在，请手动删除".format(output_path))
        self.db = h5py.File(output_path, "w")
        self.data = self.db.create_dataset(data_key, dims, dtype="float")
        self.labels = self.db.create_dataset("labels", (dims[0],), dtype="int")

        self.buf_size = buf_size
        self.buffer = {"data": [], "labels": []}
        self.idx = 0

    def add(self, raw, label):
        self.buffer["data"].extend(raw)
        self.buffer["labels"].extend(label)
        if len(self.buffer["data"]) >= self.buf_size:
            self.flush()

    def flush(self):
        i = self.idx + len(self.buffer["data"])
        self.data[self.idx:i] = self.buffer["data"]
        self.labels[self.idx:i] = self.buffer["labels"]
        self.idx = i
        self.buffer = {"data": [], "labels": []}

    def store_class_labels(self, class_labels):
        dt = h5py.special_dtype(vlen=str)
        label_dim = (len(class_labels),)
        label_set = self.db.create_dataset("label_names", label_dim, dtype=dt)
        label_set[:] = class_labels

    def close(self):
        if len(self.buffer["data"]) > 0:
            self.flush()
        self.db.close()


## HDF5DatasetGenerator

In [ ]:
from keras.utils.np_utils import to_categorical
import numpy as np
import h5py


class HDF5DatasetGenerator:
    def __init__(self, db_file, batch_size, preprocessors=None, aug=None, binarize=True, classes=2):
        self.batch_size = batch_size
        # 数据预处理器列表
        self.preprocessor = preprocessors
        # 数据增强处理器列表
        self.aug = aug
        self.binarize = binarize
        self.classes = classes
        self.db = h5py.File(db_file,'r')
        self.numImages = self.db["labels"].shape[0]

    def generator(self, passes=np.inf):
        epochs = 0

        while epochs < passes:
            for i in np.arange(0, self.numImages, self.batch_size):
                images = self.db["images"][i:i + self.batch_size]
                labels = self.db["labels"][i:i + self.batch_size]

                if self.binarize:
                    labels = to_categorical(labels, self.classes)
                if self.preprocessor is not None:
                    processed_image = []
                    for image in images:
                        for p in self.preprocessor:
                            image = p.preprocess(image)
                        processed_image.append(image)
                    images = np.array(processed_image)
                if self.aug is not None:
                    (images, labels) = next(self.aug.flow(images, labels, batch_size=self.batch_size))

                    yield images, labels
                epochs += 1

    def close(self):
        self.db.close()


## TrainingMonitor

In [ ]:
from tensorflow.keras.callbacks import BaseLogger
import matplotlib.pyplot as plt
import numpy as np
import json
import os


class TrainingMonitor(BaseLogger):
    def __init__(self,fig_path,json_path=None, start_at =0):
        super(TrainingMonitor, self).__init__()
        self.history = {}
        self.fig_path = fig_path
        self.json_path = json_path
        self.start_at = start_at

    def on_train_begin(self, logs={}):
        if self.json_path is not None:
            if os.path.exists(self.json_path):
                self.history = json.loads(open(self.json_path).read())

                if self.start_at > 0:
                    for k in self.history.keys():
                        self.history[k] = self.history[k][:self.start_at]


    def on_epoch_end(self, epoch, logs={}):
        for (k,v) in logs.items():
            log = self.history.get(k, [])
            log.append(v)
            self.history[k] =  log

        if self.json_path is not None:
            f = open(self.json_path, "w")
            f.write(json.dumps(self.history))
            f.close()


        if len(self.history["loss"]) >1:
            N = np.arange(0, len(self.history["loss"]))
            plt.style.use("ggplot")
            plt.figure()
            plt.plot(N, self.history["loss"], label="train_loss")
            plt.plot(N, self.history["val_loss"], label="val_loss")
            plt.plot(N, self.history["accuracy"], label="train_acc")
            plt.plot(N, self.history["val_accuracy"], label="val_acc")
            epochs = len(self.history["loss"])
            plt.title("Training Loss & Accuracy [Epoch {}]".format(epochs))
            plt.xlabel("Epoch #")
            plt.ylabel("Loss/Accuracy")
            plt.legend()
            plt.savefig(self.fig_path)
            plt.close()


## build_hdf5

In [ ]:
import numpy as np
# from utils.HDF5DatasetWriter import HDF5DatasetWriter
# from config import setting

print("[信息] 加载csv格式数据集文件")

file = open(SAMPLE_FILE_PATH)
file.__next__()#跳过第一行
(train_images, train_label) = ([], [])
(val_images, val_label) = ([], [])
(test_images, test_label) = ([], [])
count_by_label_train = {}
count_by_label_val = {}
count_by_label_test = {}
for row in file:
    (label, usage, image) = row.strip().split(",")
    label = int(label)
    image = np.array(image.split(" "), dtype="uint8")
    image = image.reshape((48, 48))

    if usage == "Training":
        train_images.append(image)
        train_label.append(label)
        count = count_by_label_train.get(label, 0)
        count_by_label_train[label] = count + 1

    elif usage == "PublicTest":
        val_images.append(image)
        val_label.append(label)
        count = count_by_label_val.get(label, 0)
        count_by_label_val[label] = count + 1

    elif usage == "PrivateTest":
        test_images.append(image)
        test_label.append(label)
        count = count_by_label_test.get(label, 0)
        count_by_label_test[label] = count + 1

file.close()
print("[信息] 训练集样本数量：{}".format(len(train_images)))
print("[信息] 校验集样本数量：{}".format(len(val_images)))
print("[信息] 测试集样本数量：{}".format(len(test_images)))
#训练集样本分布
print(count_by_label_train)
#校正集样本分布
print(count_by_label_val)
#测试集样本分布
print(count_by_label_test)

datasets = [(train_images,train_label,TRAIN_HDF5),
            (val_images,val_label,VAL_HDF5),
            (test_images,test_label,TEST_HDF5)]

for (images,labels,outputPath) in datasets:
    print("[信息]构建{}...".format(outputPath))
    writer = HDF5DatasetWriter((len(images),48,48),outputPath)

    for (image,label) in zip(images,labels):
        writer.add([image],[label])

    writer.close()



## mini_vgg_13

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Activation, BatchNormalization


class MiniVGG13Net():
    @staticmethod
    def build(width, height, channel, classes, reg=0.0002):
        model = Sequential(name="MiniVGG13Net")
        shape = (width, height, channel)
        channel_dimension = -1
        if backend.image_data_format == "channel first":
            shape = (channel, width, height)
            channel_dimension = 1
        # 第一卷积块
        model.add(Conv2D(64, (3, 3), input_shape=shape, padding="same"))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))

        # model.add(Conv2D(64, (3, 3), input_shape=shape, padding="same", kernel_regularizer=l2(reg)))
        # model.add(Activation("relu"))
        # model.add(Conv2D(64, (3, 3), padding="same", kernel_regularizer=l2(reg)))
        # model.add(Activation("relu"))
        # model.add(BatchNormalization(axis=channel_dimension))
        # model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
        # model.add(Dropout(0.35))

        # 第二卷积块
        model.add(Conv2D(128, (3, 3), padding="same"))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))


        # model.add(Conv2D(128, (3, 3), padding="same", kernel_regularizer=l2(reg)))
        # model.add(Activation("relu"))
        # model.add(Conv2D(128, (3, 3), padding="same", kernel_regularizer=l2(reg)))
        # model.add(Activation("relu"))
        # model.add(BatchNormalization(axis=channel_dimension))
        # model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
        # model.add(Dropout(0.35))

        # 第三卷积块
        model.add(Conv2D(256, (3, 3), padding="same"))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))
        model.add(Conv2D(256, (3, 3), padding="same"))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))


        # model.add(Conv2D(256, (3, 3), padding="same", kernel_regularizer=l2(reg)))
        # model.add(Activation("relu"))
        # model.add(Conv2D(256, (3, 3), padding="same", kernel_regularizer=l2(reg)))
        # model.add(Activation("relu"))
        # model.add(Conv2D(256, (3, 3), padding="same", kernel_regularizer=l2(reg)))
        # model.add(Activation("relu"))
        # model.add(BatchNormalization(axis=channel_dimension))
        # model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
        # model.add(Dropout(0.35))

        # 第四卷积块
        model.add(Conv2D(512, (3, 3), padding="same"))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))
        model.add(Conv2D(512, (3, 3), padding="same"))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))


        # model.add(Conv2D(512, (3, 3), padding="same", kernel_regularizer=l2(reg)))
        # model.add(Activation("relu"))
        # model.add(Conv2D(512, (3, 3), padding="same", kernel_regularizer=l2(reg)))
        # model.add(Activation("relu"))
        # model.add(Conv2D(512, (3, 3), padding="same", kernel_regularizer=l2(reg)))
        # model.add(Activation("relu"))
        # model.add(BatchNormalization(axis=channel_dimension))
        # model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
        # model.add(Dropout(0.35))

        # 第五卷积块
        model.add(Conv2D(512,(3,3),padding="same"))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))
        model.add(Dropout(0.5))

        model.add(Conv2D(512, (3, 3), padding="same"))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))
        model.add(Dropout(0.5))
        model.add(MaxPooling2D(pool_size=(2,2),padding="same",strides=(1,1)))

        # 第一全连接层
        model.add(Flatten())
        model.add(Dense(256, kernel_regularizer=l2(reg)))
        # model.add(Dense(64, kernel_regularizer=l2(reg)))

        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))
        # model.add(Dropout(0.35))
        model.add(Dropout(0.5))
        # 第二全连接层
        model.add(Flatten())
        model.add(Dense(128, kernel_regularizer=l2(reg)))
        # model.add(Dense(64, kernel_regularizer=l2(reg)))
        model.add(BatchNormalization(axis=channel_dimension))
        model.add(Activation("relu"))
        # model.add(Dropout(0.35))
        model.add(Dropout(0.5))
        # 第三全连接层
        model.add(Dense(classes, kernel_regularizer=l2(reg)))
        model.add(Activation("softmax"))

        return model


if __name__ == "__main__":
    model = MiniVGG13Net.build(48, 48, 1, 7, reg=0.0002)
    print(model.summary())




## training

In [ ]:
import matplotlib
# from config import setting
# from utils.ImageToArrayPreprocessor import ImageToArrayPreprocessor
# from utils.TrainingMonitor import TrainingMonitor
# from utils.HDF5DatasetGenerator import HDF5DatasetGenerator
# from MiniVGG13 import MiniVGG13Net
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import os
matplotlib.use("Agg")

train_aug = ImageDataGenerator(rotation_range=10,
                   zoom_range = 0.1,
                   rescale=1 / 255.0,
                   fill_mode="nearest")
val_aug = ImageDataGenerator(rescale=1/255.0)

iap = ImageToArrayPreprocesor()

train_gen = HDF5DatasetGenerator(TRAIN_HDF5,
                                 BATCH_SIZE,
                                 aug=train_aug,
                                 preprocessors=[iap],
                                 classes=NUM_CLASSES)
val_gen = HDF5DatasetGenerator(VAL_HDF5,
                                 BATCH_SIZE,
                                 aug=val_aug,
                                 preprocessors = [iap],
                                 classes=NUM_CLASSES)

opt = Adam(lr = 1e-3)
model = MiniVGG13Net.build(width=48,height=48,channel=1,classes=NUM_CLASSES)
model.compile(loss="categorical_crossentropy",optimizer=opt,metrics=["accuracy"])
json_path = os.path.sep.join([OUTPUT_PATH, "MiniVGG13.json"])
fig_path = os.path.sep.join([OUTPUT_PATH, "{}.png".format(os.getpid())])
callbacks = [TrainingMonitor(fig_path=fig_path,json_path=json_path)]
model.fit_generator(train_gen.generator(),
                    steps_per_epoch=train_gen.numImages//BATCH_SIZE,
                    validation_data=val_gen.generator(),
                    validation_steps=val_gen.numImages // BATCH_SIZE,
                    epochs=50,
                    max_queue_size=BATCH_SIZE*2,
                    callbacks=callbacks,
                    verbose=1)
print("[信息] 保存模型...")
model.save(MODEL_FILE,overwrite=True)
train_gen.close()
val_gen.close()


## evaluate


In [ ]:
# from config import setting
#from utils.HDF5DatasetGenerator import HDF5DatasetGenerator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model

testAug = ImageDataGenerator(rescale=1/255.0)
iap = ImageToArrayPreprocesor()
testGen = HDF5DatasetGenerator(TEST_HDF5,
                               BATCH_SIZE,
                               aug=testAug,
                               preprocessors=[iap],
                               classes=NUM_CLASSES)
print("[信息] 加载网络模型...")
model = load_model(MODEL_FILE)

# 评估
(loss, acc) = model.evaluate_generator(testGen.generator(),
                                     steps=testGen.numImages//BATCH_SIZE,
                                     max_queue_size=BATCH_SIZE*2)
print("[信息] 测试集准确率：{:.2f}%".format(acc*100))
testGen.close()



In [ ]:
import cv2
import imutils
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

face_detector = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
model = load_model("./model.h5")

EMOTIONS = ['Angry', 'Disgust', 'Scared', 'Harry', 'Sad', 'Surprise', 'Neutral']

# 开启摄像头
capture = cv2.VideoCapture(0)

# 持续采集摄像头图像帧
while True:
    ret, frame = capture.read()
    # 帧图像缩小并灰度化
    #print(frame)
    frame = imutils.resize(frame,width=300)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 画布
    canvas = np.zeros((240,300,3),dtype="uint8")

    frameClone = frame.copy()

    # 检测人脸
    rects = face_detector.detectMultiScale(
        gray,scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30,30),
        flags=cv2.CASCADE_SCALE_IMAGE)

    if len(rects) > 0:
        # 对检测到的多个人脸框降序排序
        rect = sorted(rects, reverse=True,
                      key=lambda x:(x[2]-x[0])*(x[3]-x[1]))[0]
        (fX, fY, fW, fH) = rect
        roi =  gray[fY:fY+fH,fX:fX+fW]
        roi = cv2.resize(roi,(48,48))
        roi = roi.astype("float") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi,axis=0)

        predicts = model.predict(roi)[0]
        label = EMOTIONS[predicts.argmax()]
        cv2.putText(frameClone,label,(fX,fY-10),
                    cv2.FONT_HERSHEY_SIMPLEX,0.5,(0, 0, 255),1, cv2.LINE_AA)
        cv2.rectangle(frameClone,(fX,fY),(fX+fW,fY+fH),
                      (0,0,255),1,cv2.LINE_AA)
        for (i,(emotion,prob)) in enumerate(zip(EMOTIONS,predicts)):
            text = "{}: {:.2f}%".format(emotion,prob*100)
            w = int (prob*300)
            cv2.rectangle(canvas,(5,(i*32)+5),(5+w,(i*32)+32),(0,0,255),-1)
            cv2.putText(
                canvas,
                text,
                (10,(i*32)+23),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (255,255,255),
                1,
                cv2.LINE_AA
            )
    cv2.imshow('Emotion Detection', frameClone)
    cv2.imshow("Result",canvas)

    if cv2.waitKey(1) == 27:
        break

capture.release()
cv2.destroyAllWindows()
